# Feature Generation

This notebook was run on an AWS SageMaker instance.  Training data is cleaned using functions from feature_generation_functions.py and the pickeled and sent to an AWS S3 bucket.  Limitations from EC2 instance required that we clean and pickle data in smaller frames of chunk size 100,000 rows.

In [1]:
import pickle_functions as pf #has io, boto3, boto3.session, _pickle, pandas
import feature_generation_functions as fgf
import pandas as pd

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
train = pd.read_csv("s3://advancedml-koch-mathur-hinkson/train.csv")

In [3]:
#citation:  https://stackoverflow.com/questions/33367142/split-dataframe-into-relatively-even-chunks-according-to-length/33368088

n = 100000  #chunk row size
train_sub_dfs = [train[i:i+n] for i in range(0,train.shape[0],n)]

[i.shape for i in train_sub_dfs]

[(100000, 45),
 (100000, 45),
 (100000, 45),
 (100000, 45),
 (100000, 45),
 (100000, 45),
 (100000, 45),
 (100000, 45),
 (100000, 45),
 (100000, 45),
 (100000, 45),
 (100000, 45),
 (100000, 45),
 (100000, 45),
 (100000, 45),
 (100000, 45),
 (100000, 45),
 (100000, 45),
 (4874, 45)]

In [4]:
sub_train_df19 = train_sub_dfs[18]
sub_train_df19_preprocessed = fgf.generate_features(sub_train_df19)
pf.write_pickle_to_s3bucket(filename='sub_train_df19_preprocessed', 
                            df=sub_train_df19_preprocessed, 
                            bucket_name='advancedml-koch-mathur-hinkson')

Split comments...Elapsed Time:  0.002 minutes
Cleaned with stopwords...Elapsed Time:  0.004 minutes
Cleaned without stopwords...Elapsed Time:  0.006 minutes
Stemmed (Porter)...Elapsed Time:  0.074 minutes
Stemmed (Lancaster)...Elapsed Time:  0.063 minutes
Created bigrams...Elapsed Time:  0.002 minutes
Calculated uppercase pct...Elapsed Time:  0.002 minutes
Count punctuation...Elapsed Time:  0.001 minutes
Count words...Elapsed Time:  0.001 minutes
Count stopwords pct...Elapsed Time:  0.003 minutes
Count uppercase words...Elapsed Time:  0.002 minutes

DONE GENERATING FEATURES
Pickled and sent to bucket!


In [5]:
sub_train_df19_preprocessed.shape

(4874, 60)